In [5]:
!pip3 install geopandas


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [6]:
!pip3 install folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 2.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 3.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 4.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [13]:
import pandas as pd
import folium

# CSV 파일 읽기
data = pd.read_csv('주요관광지점 입장객 2020~2022년.csv')

# 데이터를 '총계' 기준으로 내림차순 정렬
sorted_data = data.sort_values(by='총계', ascending=False)

# 상위 13개, 다음 8개, 나머지로 데이터 나누기
top_data = sorted_data.head(13)
next_data = sorted_data.iloc[13:21]
rest_data = sorted_data.iloc[21:]

# 위도, 경도, 관광지명 추출
top_latitude = top_data['위도']
top_longitude = top_data['경도']
top_names = top_data['관광지']

next_latitude = next_data['위도']
next_longitude = next_data['경도']
next_names = next_data['관광지']

rest_latitude = rest_data['위도']
rest_longitude = rest_data['경도']
rest_names = rest_data['관광지']

# 지도 생성 (초기 위치는 첫 번째 관광지의 위치로 설정)
m = folium.Map(location=[top_latitude.iloc[0], top_longitude.iloc[0]], zoom_start=10)

# 상위 13개 관광지 위치에 마커 추가 (빨간색)
for lat, lon, name in zip(top_latitude, top_longitude, top_names):
    folium.Marker([lat, lon], popup=name, icon=folium.Icon(color='red')).add_to(m)

# 다음 8개 관광지 위치에 마커 추가 (파란색)
for lat, lon, name in zip(next_latitude, next_longitude, next_names):
    folium.Marker([lat, lon], popup=name, icon=folium.Icon(color='blue')).add_to(m)

# 나머지 관광지 위치에 마커 추가 (초록색)
for lat, lon, name in zip(rest_latitude, rest_longitude, rest_names):
    folium.Marker([lat, lon], popup=name, icon=folium.Icon(color='green')).add_to(m)

# 지도 표시
m

In [31]:
import pandas as pd
import folium
import geopandas as gpd

# 관광지 데이터 로드
tourist_data = pd.read_csv('./주요관광지점 입장객 2020~2022년.csv')

# 데이터 정렬 및 분류
sorted_tourist_data = tourist_data.sort_values(by='총계', ascending=False)
top_data = sorted_tourist_data.head(13)
next_data = sorted_tourist_data.iloc[13:21]
rest_data = sorted_tourist_data.iloc[21:]

# 지도 생성 (첫 번째 관광지 위치를 중심으로)
m = folium.Map(location=[top_data['위도'].iloc[0], top_data['경도'].iloc[0]], zoom_start=10)

# 관광지 마커 추가 함수
def add_markers(data, map_object, color):
    for _, row in data.iterrows():
        folium.Marker(
            [row['위도'], row['경도']], 
            popup=row['관광지'], 
            icon=folium.Icon(color=color)
        ).add_to(map_object)

# 마커 추가
add_markers(top_data, m, 'red')
add_markers(next_data, m, 'blue')
add_markers(rest_data, m, 'green')

# GeoJSON 파일 경로
geojson_files = {
    "97.geojson": "./97.geojson",
    "1131.geojson": "./1131.geojson",
    "1132.geojson": "./1132.geojson",
    "1135.geojson": "./1135.geojson",
    "1136.geojson": "./1136.geojson",
    "1139.geojson": "./1139.geojson"
}

# GeoJSON 데이터 로드
geojson_data = {key: gpd.read_file(path) for key, path in geojson_files.items()}

# GeoJSON 데이터를 지도에 추가하는 함수
def add_geojson_to_map(geojson, map_object, color):
    folium.GeoJson(
        geojson,
        name='geojson',
        style_function=lambda x: {'color': color},
        show=True # 기본적으로 포인트를 숨김
    ).add_to(map_object)

# 각 파일에 대한 색상 정의
geojson_colors = {
    "1132.geojson": "blue",
    "1136.geojson": "green",
    "97.geojson": "red",
    "1131.geojson": "red",
    "1135.geojson": "red",
    "1139.geojson": "red"
}

# GeoJSON 데이터 추가
for file, color in geojson_colors.items():
    add_geojson_to_map(geojson_data[file], m, color)

# 지도 표시
m
